In [ ]:
import numpy as np

# train.ipynb

## dqn agent

In [ ]:
# Define the DQN model class
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)  # Adjust layer sizes as needed
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, output_dim)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define the agent class
class ParlaysDQNAgent:
    def __init__(self, state_size, action_size, batch_size=64, gamma=0.99,
                 epsilon_start=0.0, epsilon_end=0.0, epsilon_decay=1.0,
                 learning_rate=0.001, target_update=10, memory_size=10000,
                 logging_level=logging.INFO):
        self.state_size = state_size
        self.action_size = action_size
        self.batch_size = batch_size
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon_start  # Exploration rate (0 for inference)
        self.epsilon_min = epsilon_end
        self.epsilon_decay = epsilon_decay
        self.learning_rate = learning_rate
        self.target_update = target_update  # Episodes between target network updates
        self.memory = deque(maxlen=memory_size)
        self.bankroll = 10000  # Starting bankroll
        
        # Neural networks
        self.policy_net = DQN(state_size, action_size)
        self.target_net = DQN(state_size, action_size)
        self.update_target_net()
        
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.learning_rate)
        self.criterion = nn.MSELoss()
        
        # Set up logging
        self.logger = logging.getLogger('ParlaysDQNAgent')
        self.logger.setLevel(logging_level)
        handler = logging.StreamHandler()
        handler.setLevel(logging_level)
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        if not self.logger.handlers:
            self.logger.addHandler(handler)
        self.logger.propagate = False  # Prevent duplicate logs
        
    def update_target_net(self):
        self.target_net.load_state_dict(self.policy_net.state_dict())
        
    def remember(self, state, action, reward, next_state, done):
        pass  # Not used during inference
        
    def act(self, state, available_actions):
        # Since epsilon is 0 during inference, we always exploit
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            q_values = self.policy_net(state_tensor)
        q_values = q_values.detach().numpy()[0]
        # Mask unavailable actions if necessary
        masked_q_values = q_values.copy()
        # Select the action with the highest Q-value
        action_idx = np.argmax(masked_q_values)
        return action_idx, q_values
        
    def replay(self):
        pass  # Not used during inference
        
    def decay_epsilon(self):
        pass  # Not used during inference
        
    def save_model(self, filepath):
        torch.save(self.policy_net.state_dict(), filepath)
        
    def load_model(self, filepath):
        self.policy_net.load_state_dict(torch.load(filepath))
        self.update_target_net()
        
    def american_to_decimal_odds(self, odds):
        # Convert American odds to decimal odds
        if odds > 0:
            return (odds / 100) + 1
        else:
            return (100 / abs(odds)) + 1

# Helper functions
def odds_to_implied_prob(odds):
    # Convert American odds to implied probability
    if odds > 0:
        return 100 / (odds + 100)
    else:
        return abs(odds) / (abs(odds) + 100)

def get_state(games_data, bankroll, max_games=15):
    # Initialize state with zeros and pad for games < max_games
    state = np.zeros((max_games, 2, 4))  # [Games, Teams, Features]
    
    game_ids = games_data['GAME-ID'].unique()
    for i, game_id in enumerate(game_ids):
        if i >= max_games:
            break  # Only consider up to max_games
        
        game_df = games_data[games_data['GAME-ID'] == game_id]
        teams = game_df['TEAM'].unique()
        
        for j, team in enumerate(teams):
            team_data = game_df[game_df['TEAM'] == team].iloc[0]
            # Predicted probabilities
            ml_prob = team_data['ml_prob']
            spread_prob = team_data['spread_prob']
            # Offered odds
            ml_odds = team_data['MONEYLINE']
            spread_odds = team_data['SPREAD_LINE']
            # Implied probabilities
            ml_imp_prob = odds_to_implied_prob(ml_odds)
            spread_imp_prob = odds_to_implied_prob(spread_odds)
            # Value indicators
            ml_value = ml_prob - ml_imp_prob
            spread_value = spread_prob - spread_imp_prob
            # Assign features
            state[i, j, :] = [ml_prob, ml_value, spread_prob, spread_value]
    
    state = state.flatten()
    state = np.append(state, bankroll / 10000)  # Normalize bankroll
    return state

def generate_actions(games_data, stake_sizes=[0.01, 0.02, 0.03, 0.04, 0.05]):
    actions = []
    game_ids = games_data['GAME-ID'].unique()
    for i, game_id in enumerate(game_ids):
        game_df = games_data[games_data['GAME-ID'] == game_id]
        teams = game_df['TEAM'].unique()
        for team in teams:
            for market in ['ML', 'Spread']:
                line_col = 'MONEYLINE' if market == 'ML' else 'SPREAD_LINE'
                for stake in stake_sizes:
                    action = {
                        'game_index': i,
                        'team': team,
                        'market': market,
                        'stake': stake,
                        'line': game_df[game_df['TEAM'] == team][line_col].values[0]
                    }
                    actions.append(action)
    return actions

# Load the trained agent and perform inference

# Step 1: Load the trained agent
# Replace 'state_size' and 'action_size' with the values used during training

state_size = (15 * 2 * 4) + 1  # 15 games, 2 teams, 4 features per team, plus bankroll

# We need to determine the action size as used during training
# For that, we need to generate actions for a sample data to get the action size

# Sample data to determine action size
sample_data = {
    'DATE': ['2024-11-16'] * 30,
    'GAME-ID': [i // 2 for i in range(30)],
    'TEAM': ['Team' + str(i) for i in range(30)],
    'ml_prob': np.random.rand(30),
    'ml_result': np.random.choice([True, False], 30),
    'MONEYLINE': np.random.randint(-200, 200, 30),
    'spread_prob': np.random.rand(30),
    'spread_result': np.random.choice([True, False], 30),
    'SPREAD_LINE': np.random.randint(-110, 110, 30),
}
sample_df = pd.DataFrame(sample_data)
actions = generate_actions(sample_df)
action_size = len(actions)

# Create the agent
agent = ParlaysDQNAgent(
    state_size=state_size,
    action_size=action_size,
    batch_size=64,
    gamma=0.99,
    epsilon_start=0.0,  # Set epsilon to 0 for inference
    epsilon_end=0.0,
    epsilon_decay=1.0,
    learning_rate=0.001,
    target_update=10,
    memory_size=10000,
    logging_level=logging.INFO
)

# Load the trained model weights
agent.load_model('parlays_dqn_model.pth')  # Replace with the path to your saved model

# Step 2: Prepare today's game data and state

# Replace 'today_games.csv' with your actual data file or DataFrame
# Ensure that the DataFrame has all required columns and correct data types



## model

In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score
from sklearn.utils import resample  # Added import for resample
import joblib

class BootstrapCalibratedClassifier:
    def __init__(self, n_bootstrap_samples=8, base_model_params=None, gpu_id=0):
        """
        Initializes the BootstrapCalibratedClassifier.

        Parameters:
        - n_bootstrap_samples (int): Number of bootstrap samples/models.
        - base_model_params (dict): Parameters for the XGBoost base model.
        - gpu_id (int): GPU device ID to use.
        """
        self.n_bootstrap_samples = n_bootstrap_samples
        self.bootstrap_models = []
        self.calibrated_models = []
        self.gpu_id = gpu_id  # GPU device ID

        # Default XGBoost parameters with GPU support
        self.base_model_params = base_model_params if base_model_params else {
            'tree_method': "hist",
            'device': 'cuda',
            'enable_categorical': True,
            'max_depth': 10,
            'learning_rate': 0.09937420876401226,
            'n_estimators': 12,
            'gamma': 0.340466985869831,
            'subsample': 0.7222619026651159,
            'colsample_bytree': 0.5739321839530654,
            'reg_alpha': 0.9462720081810914,
            'reg_lambda': 0.5567871265347748
            }

    def fit(self, X_train, y_train):
        """
        Fits the bootstrap and calibrated models on the training data.

        Parameters:
        - X_train (array-like): Training features.
        - y_train (array-like): Training labels.
        """
        for i in range(self.n_bootstrap_samples):
            print(f"Training bootstrap model {i+1}/{self.n_bootstrap_samples} on GPU {self.gpu_id}...")
            
            # Bootstrap sample generation
            X_train_sample, y_train_sample = resample(X_train, y_train, n_samples=len(X_train), replace=True)
            
            # Train base model
            model = xgb.XGBClassifier(**self.base_model_params)
            model.fit(X_train_sample, y_train_sample)
            self.bootstrap_models.append(model)
            
            # Calibrate the model
            calibrated_model = CalibratedClassifierCV(model, method='sigmoid', cv='prefit')
            calibrated_model.fit(X_train_sample, y_train_sample)
            self.calibrated_models.append(calibrated_model)
            print(f"Bootstrap model {i+1} trained and calibrated.")

    def predict(self, X):
        """
        Predicts class labels for samples in X.

        Parameters:
        - X (array-like): Input features.

        Returns:
        - array-like: Predicted class labels.
        """
        if not self.bootstrap_models:
            raise Exception("The model has not been fitted yet.")
        
        # Aggregate predictions from bootstrap models
        preds = np.array([model.predict(X) for model in self.bootstrap_models])
        return np.round(np.mean(preds, axis=0)).astype(int)

    def predict_proba(self, X):
        """
        Predicts class probabilities for samples in X.

        Parameters:
        - X (array-like): Input features.

        Returns:
        - array-like: Predicted class probabilities.
        """
        if not self.calibrated_models:
            raise Exception("The model has not been fitted yet.")
        
        # Aggregate calibrated predicted probabilities from bootstrap models
        proba = np.array([calibrated_model.predict_proba(X) for calibrated_model in self.calibrated_models])
        # Average probabilities across all bootstrap models
        return np.mean(proba, axis=0)

    def score(self, X_test, y_test):
        """
        Calculates the accuracy score of the model.

        Parameters:
        - X_test (array-like): Test features.
        - y_test (array-like): True labels.

        Returns:
        - float: Accuracy score.
        """
        predictions = self.predict(X_test)
        return accuracy_score(y_test, predictions)

    def save_model(self, filename):
        """
        Saves the trained models to files.

        Parameters:
        - filename (str): Base filename to save the models.
        """
        for idx, model in enumerate(self.bootstrap_models):
            model_path = f"{filename}_model_{idx}.pkl"
            joblib.dump(model, model_path)
        
        # Save calibrated models
        for idx, calibrated_model in enumerate(self.calibrated_models):
            calibrated_model_path = f"{filename}_calibrated_{idx}.pkl"
            joblib.dump(calibrated_model, calibrated_model_path)
        print(f"Models saved to {filename}_model_*.pkl and {filename}_calibrated_*.pkl")

    def load_model(self, filename):
        """
        Loads both base and calibrated models from files.

        Parameters:
        - filename (str): Base filename from which to load the models.
        """
        self.bootstrap_models = []
        self.calibrated_models = []
        
        for idx in range(self.n_bootstrap_samples):
            base_model_path = f"{filename}_model_{idx}.pkl"
            calibrated_model_path = f"{filename}_calibrated_{idx}.pkl"
            
            try:
                base_model = joblib.load(base_model_path)
                calibrated_model = joblib.load(calibrated_model_path)
            except FileNotFoundError as e:
                print(f"Error loading model {idx}: {e}")
                continue
            
            self.bootstrap_models.append(base_model)
            self.calibrated_models.append(calibrated_model)
        print(f"Models loaded from {filename}_model_*.pkl and {filename}_calibrated_*.pkl")


# infer

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
import logging
from collections import deque
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Define the DQN model class
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)  # Adjust layer sizes as needed
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, output_dim)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define the agent class
class ParlaysDQNAgent:
    def __init__(self, state_size, action_size, batch_size=64, gamma=0.99,
                 epsilon_start=0.0, epsilon_end=0.0, epsilon_decay=1.0,
                 learning_rate=0.001, target_update=10, memory_size=10000,
                 logging_level=logging.INFO):
        self.state_size = state_size
        self.action_size = action_size
        self.batch_size = batch_size
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon_start  # Exploration rate (0 for inference)
        self.epsilon_min = epsilon_end
        self.epsilon_decay = epsilon_decay
        self.learning_rate = learning_rate
        self.target_update = target_update  # Episodes between target network updates
        self.memory = deque(maxlen=memory_size)
        self.bankroll = 10000  # Starting bankroll
        
        # Neural networks
        self.policy_net = DQN(state_size, action_size)
        self.target_net = DQN(state_size, action_size)
        self.update_target_net()
        
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.learning_rate)
        self.criterion = nn.MSELoss()
        
        # Set up logging
        self.logger = logging.getLogger('ParlaysDQNAgent')
        self.logger.setLevel(logging_level)
        handler = logging.StreamHandler()
        handler.setLevel(logging_level)
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        if not self.logger.handlers:
            self.logger.addHandler(handler)
        self.logger.propagate = False  # Prevent duplicate logs
        
    def update_target_net(self):
        self.target_net.load_state_dict(self.policy_net.state_dict())
        
    def remember(self, state, action, reward, next_state, done):
        pass  # Not used during inference
        
    def act(self, state, available_actions):
        # Since epsilon is 0 during inference, we always exploit
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            q_values = self.policy_net(state_tensor)
        q_values = q_values.detach().numpy()[0]
        # Mask unavailable actions if necessary
        masked_q_values = q_values.copy()
        # Select the action with the highest Q-value
        action_idx = np.argmax(masked_q_values)
        return action_idx, q_values
        
    def replay(self):
        pass  # Not used during inference
        
    def decay_epsilon(self):
        pass  # Not used during inference
        
    def save_model(self, filepath):
        torch.save(self.policy_net.state_dict(), filepath)
        
    def load_model(self, filepath):
        self.policy_net.load_state_dict(torch.load(filepath))
        self.update_target_net()
        
    def american_to_decimal_odds(self, odds):
        # Convert American odds to decimal odds
        if odds > 0:
            return (odds / 100) + 1
        else:
            return (100 / abs(odds)) + 1

# Helper functions
def odds_to_implied_prob(odds):
    # Convert American odds to implied probability
    if odds > 0:
        return 100 / (odds + 100)
    else:
        return abs(odds) / (abs(odds) + 100)

def get_state(games_data, bankroll, max_games=15):
    # Initialize state with zeros and pad for games < max_games
    state = np.zeros((max_games, 2, 4))  # [Games, Teams, Features]
    
    game_ids = games_data['GAME-ID'].unique()
    for i, game_id in enumerate(game_ids):
        if i >= max_games:
            break  # Only consider up to max_games
        
        game_df = games_data[games_data['GAME-ID'] == game_id]
        teams = game_df['TEAM'].unique()
        
        for j, team in enumerate(teams):
            team_data = game_df[game_df['TEAM'] == team].iloc[0]
            # Predicted probabilities
            ml_prob = team_data['ml_prob']
            spread_prob = team_data['spread_prob']
            # Offered odds
            ml_odds = team_data['MONEYLINE']
            spread_odds = team_data['SPREAD_LINE']
            # Implied probabilities
            ml_imp_prob = odds_to_implied_prob(ml_odds)
            spread_imp_prob = odds_to_implied_prob(spread_odds)
            # Value indicators
            ml_value = ml_prob - ml_imp_prob
            spread_value = spread_prob - spread_imp_prob
            # Assign features
            state[i, j, :] = [ml_prob, ml_value, spread_prob, spread_value]
    
    state = state.flatten()
    state = np.append(state, bankroll / 10000)  # Normalize bankroll
    return state

def generate_actions(games_data, stake_sizes=[0.01, 0.02, 0.03, 0.04, 0.05]):
    actions = []
    game_ids = games_data['GAME-ID'].unique()
    for i, game_id in enumerate(game_ids):
        game_df = games_data[games_data['GAME-ID'] == game_id]
        teams = game_df['TEAM'].unique()
        for team in teams:
            for market in ['ML', 'Spread']:
                for stake in stake_sizes:
                    action = {
                        'game_index': i,
                        'team': team,
                        'market': market,
                        'stake': stake
                    }
                    actions.append(action)
    return actions

# Load the trained agent and perform inference

# Step 1: Load the trained agent
# Replace 'state_size' and 'action_size' with the values used during training

state_size = (15 * 2 * 4) + 1  # 15 games, 2 teams, 4 features per team, plus bankroll

# We need to determine the action size as used during training
# For that, we need to generate actions for a sample data to get the action size

# Sample data to determine action size
sample_data = {
    'DATE': ['2024-11-16'] * 30,
    'GAME-ID': [i // 2 for i in range(30)],
    'TEAM': ['Team' + str(i) for i in range(30)],
    'ml_prob': np.random.rand(30),
    'ml_result': np.random.choice([True, False], 30),
    'MONEYLINE': np.random.randint(-200, 200, 30),
    'spread_prob': np.random.rand(30),
    'spread_result': np.random.choice([True, False], 30),
    'SPREAD_LINE': np.random.randint(-110, 110, 30),
}
sample_df = pd.DataFrame(sample_data)
actions = generate_actions(sample_df)
action_size = len(actions)

# Create the agent
agent = ParlaysDQNAgent(
    state_size=state_size,
    action_size=action_size,
    batch_size=64,
    gamma=0.99,
    epsilon_start=0.0,  # Set epsilon to 0 for inference
    epsilon_end=0.0,
    epsilon_decay=1.0,
    learning_rate=0.001,
    target_update=10,
    memory_size=10000,
    logging_level=logging.INFO
)

# Load the trained model weights
agent.load_model('parlays_dqn_model_1.pth')  # Replace with the path to your saved model

# Step 2: Prepare today's game data and state

# Replace 'today_games.csv' with your actual data file or DataFrame
# Ensure that the DataFrame has all required columns and correct data types
today_games_df = today_results
today_games_df['SPREAD_LINE'] = -110
# Ensure required columns are present
required_columns = [
    'DATE', 'GAME-ID', 'TEAM', 'ml_prob', 'MONEYLINE',
    'spread_prob', 'SPREAD_LINE'
]
if not all(col in today_games_df.columns for col in required_columns):
    raise ValueError("Missing required columns in today's game data.")

# Set the agent's bankroll
agent.bankroll = 100  # Set your current bankroll

# Prepare the state
state = get_state(today_games_df, agent.bankroll, max_games=15)

# Step 3: Generate available actions
actions = generate_actions(today_games_df)
action_size = len(actions)
available_actions = list(range(action_size))

# Step 4: Use the agent to select actions
action_idx, q_values = agent.act(state, available_actions)
action = actions[action_idx]

# If you wish to select the top N actions
N = 10  # Number of top actions to select
top_N_action_indices = np.argsort(q_values)[-N:][::-1]

# Step 5: Interpret and output the selected actions
for idx in top_N_action_indices:
    action = actions[idx]
    stake_amount = action['stake'] * agent.bankroll
    team = action['team']
    market = action['market']
    game_index = action['game_index']
    
    # Get the actual game ID for clarity
    game_ids = today_games_df['GAME-ID'].unique()
    if game_index < len(game_ids):
        game_id = game_ids[game_index]
    else:
        game_id = 'Unknown'
    
    print(f"Selected Action (Q-value: {q_values[idx]:.2f}):")
    print(f"Bet ${stake_amount:.2f} on {team} in game {game_id}, market: {market}")
    print()


## MIP PArlays

In [ ]:
from pulp import *

# Create the problem instance
model = LpProblem("Optimal_Parlays", LpMaximize)

# Example data
legs = range(20)  # 10 legs available
probabilities = {l: np.random.uniform(0.2, 0.8) for l in legs}  # Probabilities for each leg
odds = {l: 2.0 for l in legs}  # Example odds (flat 2x payout)
opposing_pairs = [(0, 1), (2, 3)]  # Define opposing bets
parlays = range(10)  # Build up to 5 parlays
alpha = 10  # Scaling factor for exposure
k = 7  # Number of legs per parlay

wager_per_parlay = 10  # Fixed wager per parlay
budget = len(parlays) * wager_per_parlay  # Total budget for all parlays
# Decision variables
x = LpVariable.dicts("x", [(p, l) for p in parlays for l in legs], cat="Binary")
y = LpVariable.dicts("y", parlays, cat="Binary")

# Objective: Maximize expected profit
model += lpSum(
    y[p] * lpSum(probabilities[l] * (odds[l] - 1) for l in legs if x[p, l])
    for p in parlays
)

# Constraints

# Each parlay has exactly k legs
for p in parlays:
    model += lpSum(x[p, l] for l in legs) == k

# Exposure constraint proportional to probability
for l in legs:
    model += lpSum(x[p, l] for p in parlays) <= alpha * probabilities[l]

# Budget constraint
model += lpSum(y[p] * wager_per_parlay for p in parlays) <= budget

# Leg inclusion only if parlay is selected
for p in parlays:
    for l in legs:
        model += x[p, l] <= y[p]

# Opposing bets constraint
for p in parlays:
    for (l1, l2) in opposing_pairs:
        model += x[p, l1] + x[p, l2] <= 1

# Solve the model
model.solve(PULP_CBC_CMD())

# Output results
print("Status:", LpStatus[model.status])
for p in parlays:
    if y[p].varValue > 0:
        selected_legs = [l for l in legs if x[p, l].varValue > 0]
        print(f"Parlay {p}: Legs {selected_legs}")


#  nba_infer CODE

In [ ]:
print('NORMALIZED PREDICTED WINNERS\n')
display(today_results[(today_results['spread_prob_normed'] > 0.5) & (today_results['spread_prob'] > 0.5)] \
.sort_values('spread_prob', ascending=False)[['TEAM', 'Opponent','CLOSING_SPREAD',
                                                'spread_prob', 'spread_prob_normed', 'spread_kelly', 'spread_implied_prob']].drop_duplicates(subset=['TEAM']))

print('\nALL PREDICTED WINNERS\n')
today_results[(today_results['spread_prob_normed'] > 0.5)] \
.sort_values('spread_prob', ascending=False)[['TEAM', 'Opponent','CLOSING_SPREAD',
                                                'spread_prob', 'spread_prob_normed']].drop_duplicates(subset=['TEAM'])


In [ ]:
print('NORMALIZED PREDICTED WINNERS\n')
display(today_results[(today_results['ml_prob_normed'] > 0.5) & (today_results['ml_prob'] > 0.5)] \
.sort_values('ml_prob', ascending=False)[['TEAM', 'Opponent','MONEYLINE',
                                                'ml_prob', 'ml_prob_normed']].drop_duplicates(subset=['TEAM']))

print('ALL PREDICTED WINNERS\n')
display(today_results[(today_results['ml_prob_normed'] > 0.5)] \
.sort_values('ml_prob', ascending=False)[['TEAM', 'Opponent','MONEYLINE',
                                                'ml_prob', 'ml_prob_normed']].drop_duplicates(subset=['TEAM']))


In [ ]:
over_df['lambda'] = over_df.apply(
    lambda row: solve_poisson_for_over(
        k=int(row['CLOSING_TOTAL']), 
        p=float(row['total_prob'])
    ),
    axis=1
)

# Now df["lambda"] holds the Poisson λ that gives P(X >= TOTAL) = prob for each row.
over_df[['TEAM', 'Opponent','CLOSING_TOTAL', 'total_prob', 'lambda']]

In [ ]:
import math

def logaddexp(a, b):
    """
    Equivalent to math.log(math.exp(a) + math.exp(b)), 
    but numerically stable for large or very negative values.
    """
    if a == b:
        return a + math.log(2)
    elif a > b:
        return a + math.log1p(math.exp(b - a))
    else:
        return b + math.log1p(math.exp(a - b))


def log_poisson_pmf(lam, i):
    """
    log( P(X=i) ) for a Poisson(lam), i.e.:
    log( e^(-lam) * lam^i / i! )
    = -lam + i*log(lam) - logGamma(i+1)
    """
    return -lam + i * math.log(lam) - math.lgamma(i+1)

def log_poisson_cdf(lam, k):
    """
    Returns log( P(X <= k) ) for X ~ Poisson(lam).
    We do a sum of pmf(i) from i=0..k, all in log-space:
        sum_{i=0..k} exp( log_poisson_pmf(lam, i) )
    and keep a running log-sum with math.logaddexp to avoid overflow.
    """
    log_sum = float('-inf')
    for i in range(k+1):
        lp = log_poisson_pmf(lam, i)
        # logaddexp(a, b) = log( exp(a) + exp(b) )
        log_sum = logaddexp(log_sum, lp)
    return log_sum  # This is log of the actual CDF

def solve_poisson_for_over(k, p, lower=1e-9, upper=3000, tol=1e-7):
    """
    Find λ so that P(X >= k) = p for X ~ Poisson(λ),
    which means P(X <= k-1) = 1 - p.

    - k: integer threshold
    - p: desired probability P(X >= k)
    - lower, upper: bracket for λ
    - tol: numerical tolerance

    Returns: λ that solves Poisson(λ) with P(X >= k)=p,
             or raises ValueError if no root in [lower, upper].
    """
    # We want CDF(k-1) = 1 - p
    target_cdf = 1 - p
    
    def f(lam):
        # log(CDF) for X <= k-1
        log_cdf_val = log_poisson_cdf(lam, k-1)  # log of sum_{i=0..k-1} pmf(i)
        cdf_val = math.exp(log_cdf_val)
        return cdf_val - target_cdf
    
    # Check boundary signs
    f_low = f(lower)
    f_high = f(upper)
    if abs(f_low) < 1e-15:
        return lower
    if abs(f_high) < 1e-15:
        return upper
    if f_low * f_high > 0:
        raise ValueError("No sign change in [lower, upper]; adjust bounds.")
    
    # Bisection loop
    while (upper - lower) > tol:
        mid = 0.5*(lower + upper)
        fm = f(mid)
        if fm == 0:
            return mid
        if fm * f_low < 0:
            upper = mid
            f_high = fm
        else:
            lower = mid
            f_low = fm
    return 0.5 * (lower + upper)

# Example usage
if __name__ == "__main__":
    # Suppose we want P(X >= 210) = 0.8
    threshold = 230
    prob = 0.97
    lam_est = solve_poisson_for_over(threshold, prob)
    print(f"λ ≈ {round(lam_est*2)/2} so that P(X >= {threshold}) = {prob} for Poisson(λ).")


# nba_2024 CODE

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Predict on x_test
y_pred = pts_model.predict(X_test.drop('GAME-ID', axis=1))

# Step 2: Create a DataFrame with predictions and actual values
pred_df = pd.DataFrame({
    'Predicted_PTS': y_pred,
    'Actual_PTS': y_test
})


# Step 3: Calculate error
pred_df['Prediction_Error'] = abs(pred_df['Predicted_PTS'] - pred_df['Actual_PTS'])

# Step 4: Check if the predicted points meet or exceed the player prop line
pred_df['Prediction_Met'] = pred_df['Predicted_PTS'] >= pred_df['Actual_PTS']

# Step 5: Define bins based on prediction magnitude (e.g., every 5 points)
pred_df['Prediction_Bin'] = pd.cut(pred_df['Predicted_PTS'], bins=10)

# Step 6: Calculate accuracy within each bin
bin_accuracy = pred_df.groupby('Prediction_Bin')['Prediction_Met'].mean().reset_index()
bin_accuracy.columns = ['Prediction_Bin', 'Accuracy']

# Step 7: Plot a histogram of accuracy by prediction magnitude bin
plt.figure(figsize=(10, 6))
bars = plt.bar(bin_accuracy['Prediction_Bin'].astype(str), bin_accuracy['Accuracy'] * 100)

# Annotate bars with actual accuracy values
for bar, accuracy in zip(bars, bin_accuracy['Accuracy']):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(),
             f'{accuracy * 100:.2f}%', ha='center', va='bottom')

# Set labels and title
plt.xticks(rotation=45)
plt.xlabel('Prediction Magnitude (Binned)')
plt.ylabel('Accuracy (%)')
plt.title('Prediction Accuracy by Prediction Magnitude')
plt.show()
